1、导入所需库文件

In [3]:
from __future__ import division

import talib
import time
import datetime

import math

import numpy as np
import pandas as pd

# import statsmodels
# from statsmodels.tsa.stattools import coint


# 引入统计函数库
# 网上此种引入方法比较普遍
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

2、读入读取的数据

In [4]:
# 根据不同的文件名，读取相应的数据
price_data = pd.read_csv('price0308.csv',index_col = 0)

# 转换为收益率数据
price_data = np.log(price_data/price_data.shift(1))*10000

# nan处理
price_data = price_data.fillna(0)

# inf处理
price_data[np.isinf(price_data)] = 0

3、寻找相应的配对股票
配对原则：Pvalue小，相关性高

In [ ]:
# 证监会行业列表
industry_list = ["A01","A02","A03","A04","A05","B06","B07","B08","B09","B10","B11", \
             "B12","C13","C14","C15","C16","C17","C18","C19","C20","C21","C22", \
             "C23","C24","C25","C26","C27","C28","C29","C30","C31","C32","C33", \
             "C34","C35","C36","C37","C38","C39","C40","C41","C42","C43","D44", \
             "D45","D46","E47","E48","E49","E50","F51","F52","G53","G54","G55", \
             "G56","G57","G58","G59","G60","H61","H62","I63","I64","I65","J66", \
             "J67","J68","J69","K70","L71","L72","M73","M74","M75","N76","N77", \
             "N78","O79","O80","O81","P82","Q83","Q84","R85","R86","R87","R88","R89","S90"]

# 开始进行行业循环，提取相应的
for industry_list_i in industry_list:

    print "开始计算：%s"%str(industry_list_i)

    # 清空存储数据
    # 用于整理数据，中间暂存每个配对对应的pearson、beta等相关信息
    data_pd = pd.DataFrame()

    # 清空存储配对数据
    pairs = []

    # 清空数据
    stock_list = []

    # 获取行业列表
    stock_list = get_industry_stocks(industry_list_i, date= current_date)

In [16]:
# 函数名：find_cointegrated_pairs
# 输入参数：
# data：行为日期数据，列为股票价格数据
# 输出参数：
# score_matrix：两两股票之间对应的相关性举证
# pvalue_matrix：两两股票之间对应的pvalue矩阵
# pairs：两两配对的股票

def find_cointegrated_pairs(data):
   
    # 返回参数
    ret_list = []
    
    n = data.shape[1]
    keys = data.keys()

    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            res = S1 - S2


            cadf = ts.adfuller(res)

            # 如果小于阈值，则可以以5%的置信度可以推翻Null hypothesis
            # null hypothesis:  there isn't a cointegrating relationship at the 5% level.
            if cadf[1] < 0.05:
                if calcPearson(S1,S2)>0.7:
                    if hurst_joinquant(res) < 0.3:
                        ret_list.append([keys[i],keys[j],cadf[1],calcPearson(S1,S2),hurst_joinquant(res)])
                        print "find pairs: %s,%s"%(str(keys[i]),str(keys[j]))
    return ret_list
            



# 函数名：hurst
# 输入参数：
# 1、ts：残差序列
# 输出参数：
# 大于0.5则说明是趋势，小于0.5均值回复
def hurst(ts):
    """Returns the Hurst Exponent of the time series vector ts"""
    # Create the range of lag values
    lags = range(2, 100)

    # Calculate the array of the variances of the lagged differences
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]

    # Use a linear fit to estimate the Hurst Exponent
    poly = polyfit(log(lags), log(tau), 1)

    # Return the Hurst exponent from the polyfit output
    return poly[0]*2.0


#计算Pearson系数
def calcPearson(x,y):
    #计算特征和类的平均值
    def _calcMean(x,y):
        sum_x = sum(x)
        sum_y = sum(y)
        
        n = len(x)
        if n != 0:
            x_mean = float(sum_x+0.0)/n
            y_mean = float(sum_y+0.0)/n
            return x_mean,y_mean
        else:
            return 1,1
    x_mean,y_mean = _calcMean(x,y)	#计算x,y向量平均值
    n = len(x)
    sumTop = 0.0
    sumBottom = 0.0
    x_pow = 0.0
    y_pow = 0.0
    for i in range(n):
        sumTop += (x[i]-x_mean)*(y[i]-y_mean)
    for i in range(n):
        x_pow += math.pow(x[i]-x_mean,2)
    for i in range(n):
        y_pow += math.pow(y[i]-y_mean,2)
    sumBottom = math.sqrt(x_pow*y_pow)
    if sumBottom != 0:
        p = sumTop/sumBottom
        return p
    else:
        return 0

#6 R/S算法计算Hurst
#变量：x-日收益率-list(之后使用np.array变换成数组) 
#输出：计算得到的Hurst值
def hurst_joinquant(X):
    
    #输入日回报率
    X = np.array(X)
    
    #N代表最大的片段值（即不对X分割时）
    N = X.size
    
    T = np.arange(1, N + 1)
    Y = np.cumsum(X)
    
    #分别计算不同长度的片段的均值
    Ave_T = Y / T

    #每个片段的最大差距R_T
    R_T = np.zeros(N)
    #对应的每段的标准差S_T
    S_T = np.zeros(N)
    
    #分别对不同的大小的切片计算R_T和S_T
    for i in range(N):
        S_T[i] = np.std(X[:i + 1])
        Z_T = Y - T * Ave_T[i]
        R_T[i] = np.ptp(Z_T[:i + 1])
        
    #计算R/S
    R_S = R_T / S_T
    
    #将lg(R/S)作为被解释变量Y
    R_S = np.log(R_S)[1:]
    
    #将lgt作为解释变量X
    n = np.log(T)[1:]
    A = np.column_stack((n, np.ones(n.size)))
    
    #回归得到的斜率即为hurst指数
    [m, c] = np.linalg.lstsq(A, R_S)[0]
    H = m
    return H

In [15]:
pairs = []
pairs = find_cointegrated_pairs(price_data)

c:\python27\lib\site-packages\ipykernel_launcher.py:120: RuntimeWarning: invalid value encountered in true_divide


find pairs: 000728.XSHE,600109.XSHG


In [17]:
pairs

[['000728.XSHE',
  '600109.XSHG',
  2.0511864153625317e-30,
  0.91717569780957686,
  0.29367388710380665]]

In [16]:
# 函数名：find_cointegrated_pairs
# 输入参数：
# data：行为日期数据，列为股票价格数据
# 输出参数：
# score_matrix：两两股票之间对应的相关性举证
# pvalue_matrix：两两股票之间对应的pvalue矩阵
# pairs：两两配对的股票
i = 1
j = 15
keys = data.keys()

data = price_data
S1 = data[keys[i]]
S2 = data[keys[j]]
res = S1 - S2


cadf = ts.adfuller(res)
    
# 如果小于阈值，则可以以5%的置信度可以推翻Null hypothesis
# null hypothesis:  there isn't a cointegrating relationship at the 5% level.
if cadf[0] < -2.873608704758507:
    print "ok"
else:
    print "not ok"

ok


4、存储相应的数据

In [ ]:
# 存储scores
np.savetxt('scores.csv',scores,delimiter=',')

# 存储pvalues
np.savetxt('pvalues.csv',pvalues,delimiter=',')

# 存储相应的配对股票数据
pairs_pd = pd.DataFrame(columns=['pairs1','pairs2','residual','Pvalue','correlation'],data=pairs)
pairs_pd.to_csv("pairs.csv")

In [5]:
# 存储Z_score数据
score_pd= pd.DataFrame()

# 转换收盘价数据为Z_score数据
score_pd = data_transfer(price_data)
score_pd.to_csv('z_score.csv')

c:\python27\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in true_divide


In [58]:
Pairs_Info = pd.read_csv('pairs.csv',index_col = 0)
Pairs_Info['pairs1_name'] = None
Pairs_Info['pairs2_name'] = None


Pairs_Info['pairs1_industry'] = None
Pairs_Info['pairs2_industry'] = None

# 逐列开始添加信息
for i in range(0,Pairs_Info.shape[0]):
    print i
    Pairs_Info



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [4]:
# 功能，将输入的数据转化为长期的监测数据
# 公式：z score: (5d MA — 60d MA) /60d SD
# 输入：
# price_data：日期作为index，以股票名称作为列，其中的数据内容是收盘价
# 输出：
# 转换后的数据
# z score: (5d MA — 60d MA) /60d SD
# 因为是60天的数据，为保证数据准确进行了截取，只取100天以后的数据

def data_transfer(price_data):
    
    # 存储数据
    df_stocks = pd.DataFrame(index = np.array(price_data.index)[100:])

    # 获取编码
    stocks = price_data.keys()
    
    # 计算z score: (5d MA — 60d MA) /60d SD
    for stock_name in stocks:
        
        sma_5 = talib.SMA(np.array(price_data[stock_name]), timeperiod=5)
        sma_60 = talib.SMA(np.array(price_data[stock_name]), timeperiod=60)
        std_60 = talib.STDDEV(np.array(price_data[stock_name]), timeperiod=60)

        z_value = (sma_5 - sma_60)/std_60

        df_stocks = pd.concat([df_stocks,pd.DataFrame(data=np.array(z_value[100:]),index=np.array(price_data[stock_name].index)[100:],columns = [stock_name])],axis = 1)
    
    # nan处理
    df_stocks = df_stocks.fillna(0)
    # inf处理
    df_stocks[np.isinf(df_stocks)] = 0
    
    return df_stocks
